<center><h1 style="color:green">Tensorflow Input Pipeline</center>

In [1]:
import tensorflow as tf

<b>Create tf dataset from a list

In [2]:
daily_sales_numbers = [21, 22, -108, 31, -1, 32, 34,31]

tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_numbers)
tf_dataset

<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

<b>Iterate through tf dataset

In [3]:
for sales in tf_dataset:
    print(sales)

tf.Tensor(21, shape=(), dtype=int32)
tf.Tensor(22, shape=(), dtype=int32)
tf.Tensor(-108, shape=(), dtype=int32)
tf.Tensor(31, shape=(), dtype=int32)
tf.Tensor(-1, shape=(), dtype=int32)
tf.Tensor(32, shape=(), dtype=int32)
tf.Tensor(34, shape=(), dtype=int32)
tf.Tensor(31, shape=(), dtype=int32)


In [4]:
for sales in tf_dataset:
    print(sales.numpy())

21
22
-108
31
-1
32
34
31


<b>Iterate through elements as numpy elements

In [5]:
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

21
22
-108
31
-1
32
34
31


<b>Iterate through first n elements in tf dataset

In [6]:
for sales in tf_dataset.take(3):
    print(sales.numpy())

21
22
-108


<b>Filter sales numbers that are < 0

In [7]:
tf_dataset = tf_dataset.filter(lambda x: x>0)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

21
22
31
32
34
31


<b>Convert sales numbers from USA dollars ($) to Bangladeshi tk Assuming 1->112 conversation rate

In [8]:
tf_dataset = tf_dataset.map(lambda x: x*112)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

2352
2464
3472
3584
3808
3472


<b>Shuffle

In [9]:
tf_dataset = tf_dataset.shuffle(2)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

2464
3472
2352
3808
3584
3472


<b>Batching

In [10]:
for sales_batch in tf_dataset.batch(2):
    print(sales_batch.numpy())

[2352 3472]
[2464 3584]
[3808 3472]


<b>Perform all of the above operations in one shot

In [11]:
tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_numbers)

tf_dataset = tf_dataset.filter(lambda x: x>0).map(lambda y: y*112).shuffle(2).batch(2)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

[2352 2464]
[3472 3584]
[3472 3808]


<b>Images

In [12]:
images_ds = tf.data.Dataset.list_files('images/*/*', shuffle=False)

In [13]:
image_count = len(images_ds)
image_count

130

In [14]:
type(images_ds)

tensorflow.python.data.ops.from_tensor_slices_op._TensorSliceDataset

In [15]:
for file in images_ds.take(3):
    print(file.numpy())

b'images\\cat\\20 Reasons Why Cats Make the Best Pets....jpg'
b'images\\cat\\7 Foods Your Cat Can_t Eat.jpg'
b'images\\cat\\A cat appears to have caught the....jpg'


In [16]:
images_ds = images_ds.shuffle(200)
for file in images_ds.take(3):
    print(file.numpy())

b'images\\dog\\Best Dog & Puppy Health Insurance Plans....jpg'
b'images\\dog\\Best Hypoallergenic Dogs [Updated....jpg'
b'images\\cat\\7 Foods Your Cat Can_t Eat.jpg'


In [17]:
class_names = ["cat","dog"]

In [18]:
train_size = int(image_count*0.8)
train_ds = images_ds.take(train_size)
test_ds = images_ds.skip(train_size)

In [19]:
len(train_ds)

104

In [20]:
len(test_ds)

26

In [21]:
def get_label(file_path):
    import os
    parts = tf.strings.split(file_path, os.path.sep)
    return parts[-2]

In [22]:
def process_image(file_path):
    label = get_label(file_path)
    img = tf.io.read_file(file_path) # load the raw data from the file as a string
    img = tf.image.decode_jpeg(img)
    img = tf.image.resize(img, [128, 128])
    return img, label

In [23]:
img, label = process_image("images\\cat\\20 Reasons Why Cats Make the Best Pets....jpg")
img.numpy()[:2]

array([[[155.      , 186.      , 215.      ],
        [156.      , 187.      , 216.      ],
        [158.      , 189.      , 218.      ],
        [160.0039  , 189.0039  , 219.0039  ],
        [161.0039  , 190.0039  , 220.0039  ],
        [162.      , 191.      , 221.      ],
        [166.      , 193.      , 222.      ],
        [167.      , 194.      , 223.      ],
        [168.      , 195.      , 224.      ],
        [169.      , 196.      , 225.      ],
        [170.      , 197.      , 224.      ],
        [170.0039  , 197.0039  , 224.0039  ],
        [172.      , 199.      , 226.      ],
        [173.0039  , 199.0039  , 224.0039  ],
        [174.0039  , 200.0039  , 225.0039  ],
        [175.0039  , 201.0039  , 226.0039  ],
        [176.      , 202.      , 227.      ],
        [177.0039  , 203.0039  , 228.0039  ],
        [177.0039  , 203.0039  , 228.0039  ],
        [179.0039  , 203.0039  , 227.0039  ],
        [180.0039  , 204.0039  , 228.0039  ],
        [180.0039  , 204.0039  , 2

In [24]:
train_ds = train_ds.map(process_image)
test_ds = test_ds.map(process_image)

In [25]:
for image, label in train_ds.take(1):
    print("****",image)
    print("****",label)

**** tf.Tensor(
[[[177.17969  111.07031   31.695312]
  [217.10938  152.60938   68.234375]
  [203.73438  135.08594   39.507812]
  ...
  [ 65.703125  58.703125  52.703125]
  [ 49.625     44.625     40.625   ]
  [ 44.671875  43.671875  38.671875]]

 [[162.44531   82.49219   11.46875 ]
  [181.5      107.25      35.96875 ]
  [183.97656  121.828125  46.335938]
  ...
  [ 70.66406   61.664062  52.664062]
  [ 80.13281   73.13281   66.36719 ]
  [ 49.664062  44.828125  38.335938]]

 [[168.32812   93.015625  14.578125]
  [193.375    137.5       70.734375]
  [195.58594  145.66406   79.390625]
  ...
  [ 60.34375   53.59375   44.34375 ]
  [ 70.05469   60.382812  51.3125  ]
  [ 39.359375  34.132812  28.15625 ]]

 ...

 [[115.32031  114.140625 109.234375]
  [146.42969  146.41406  138.85938 ]
  [ 26.        36.476562  29.59375 ]
  ...
  [232.57031  165.25      83.58594 ]
  [212.42969  140.07031   46.335938]
  [217.92969  138.55469   37.429688]]

 [[189.78906  197.65625  190.36719 ]
  [197.25     206.335

In [26]:
def scale(image, label):
    return image/255, label

In [27]:
train_ds = train_ds.map(scale)

In [28]:
for image, label in train_ds.take(5):
    print("****Image: ",image.numpy()[0][0])
    print("****Label: ",label.numpy())

****Image:  [0.31072304 0.31072304 0.31072304]
****Label:  b'dog'
****Image:  [0.27450982 0.23137255 0.20784314]
****Label:  b'cat'
****Image:  [0.14935039 0.19651932 0.14156231]
****Label:  b'cat'
****Image:  [0.7316167  0.74805355 0.75981826]
****Label:  b'dog'
****Image:  [0.7009957  0.72844666 0.5637408 ]
****Label:  b'dog'
